# Visualize Search Results
Render a space of molecules in human-readable formats

In [ ]:
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, CDSView, GroupFilter
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import CustomJS, Div
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Draw import MolDraw2DSVG, MolToImage
from pathlib import Path
from io import BytesIO
import pandas as pd
import xlsxwriter
import yaml

Configuration

In [ ]:
data_file = 'results/anolytes-220607-neutral_only-subset_ZINC15-filter_round2.1.csv'
filter_path = Path('../screen-search-space/round2.1.yml')

Output file paths

In [ ]:
graph_file = data_file[:-4] + '.html'
table_file = data_file[:-4] + '.xlsx'

## Load in the data
Read a results file, which includes the molecule identity and properties

In [ ]:
results = pd.read_csv(data_file)
print(f'Loaded {len(results)} molecules')

Get the name of the output properties

In [ ]:
redox_col = results.columns[4]
redox_name = redox_col.split("_")[0].upper()
results['redox'] = results[redox_col]
print(f'The redox property is {redox_name}')

In [ ]:
solv_col = results.columns[5]
results['solv'] = results[solv_col]

Compute the redox potential wrt Ferrocene. See [reference](https://pubs.acs.org/doi/10.1021/ct1003252)

In [ ]:
results['redox'] -= 4.988   # Convert to wrt SHE

## Compute Vector Images of Each Molecule
Something we can visualize later

In [ ]:
def print_molecule(smiles) -> str:
    """Print a molecule as an SVG
    
    Args:
        smiles (str): SMILES string of molecule to present
        atom_id (int): ID number atom to highlight
    Returns:
        (str): SVG rendering of molecule
    """
    # Compute 2D coordinates
    mol = Chem.MolFromSmiles(smiles)
    AllChem.Compute2DCoords(mol)
    
    
    # Print out an SVG
    rsvg = MolDraw2DSVG(250, 250)
    rsvg.DrawMolecule(mol)
    rsvg.FinishDrawing()
    return rsvg.GetDrawingText().strip()

In [ ]:
results['svg'] = results['smiles'].apply(print_molecule)

## Make a Pareto Plot
Something that shows each of the results and you can move around to zoom in on specific molecules

In [ ]:
output_file(graph_file)
output_notebook()

Assign each point a color based on whether it is pareto or not

In [ ]:
results['color'] = results['is_pareto'].apply(lambda x: '#CD5C5C' if x else '#2471A3')

Define the tooltip

In [ ]:
hover = HoverTool(tooltips=f"""
<style>
        .bk-tooltip>div:not(:first-child) {{display:none;}}
</style>

@svg{{safe}}</br>
<b>SMILES</b>: @smiles{{safe}} </br>
<b>{redox_name}</b>: @redox{{0.0}} V </br>
<b>G<sub>solv</sub></b>: @solv{{0.000}} kcal/mol
""")

Make the data source

In [ ]:
results_view = ColumnDataSource(results[['smiles', 'redox', 'solv', 'color', 'svg']])

Make the pareto plot

In [ ]:
p = figure(x_axis_label=redox_name + ' (V)', y_axis_label='G_solv (kcal/mol)',
           width=1200, height=800, tools='box_select,lasso_select,pan,wheel_zoom,box_zoom,reset')
p.circle(x='redox', y='solv', color='color', source=results_view)
pareto = results.query('is_pareto')
p.step(pareto['redox'], pareto[solv_col], mode='after', color='black')

p.add_tools(hover)

Create a div that will show the selected molecules

In [ ]:
div = Div(width=p.width)

init_text = '<b>Use the select tools to highlight molecules of interest</b>'
div.text = init_text

# Make the interaction that changes the selection
results_view.selected.js_on_change('indices', CustomJS(
    args={'results': results_view, 'div': div}, 
    code=f"""
const inds = cb_obj.indices;
const data = results.data;
let output = "<p>{init_text}</p>";
output += "<table>";
output += "<tr><th>SMILES</th><th>{redox_name}</th><th>G<sub>solv</sub></th><th>Image</th>";
for (let i = 0; i < inds.length; i++) {{
    const id = inds[i];
    output += "<tr>";
    output += "<td>" + data["smiles"][id] + "</td>";
    output += "<td>" + data["redox"][id].toFixed(2) + "</td>";
    output += "<td>" + data["solv"][id].toFixed(2) + "</td>";
    output += "<td>" + data["svg"][id] + "</td>";
}}
output += "</table>";
div.text = output;
"""))

In [ ]:
show(column(p, div))

## Save it in a table format
Save the main columns in an XLSX file with pictures

First, make a function to convert a SMILES string to a PNG bytestring

In [ ]:
def molecule_to_png(smiles) -> BytesIO:
    """Print a molecule as a PNG
    
    Args:
        smiles (str): SMILES string of molecule to present
    Returns:
        (str): SVG rendering of molecule
    """
    # Compute 2D coordinates
    mol = Chem.MolFromSmiles(smiles)
    AllChem.Compute2DCoords(mol)
    
    
    # Print out an SVG
    img = MolToImage(mol, size=(256, 256))
    fp = BytesIO()
    img.save(fp, format='png')
    return fp

Now, make a subset of the dataset with only the columns we intend to save

In [ ]:
to_save = results[
    ['smiles', 'molwt', redox_col, solv_col, 'is_pareto']
].rename(columns={redox_col: redox_name, solv_col: 'G_solv'})

Add a blank column to be the image

In [ ]:
to_save['image'] = ''

Start by opening the spreadsheet [using the low-level ExcelWriter class from Pandas.](https://xlsxwriter.readthedocs.io/working_with_pandas.html)

In [ ]:
Path(table_file).unlink(missing_ok=True)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(table_file, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
to_save.to_excel(writer, sheet_name='molecules', index=False)

Fill in the last column of the sheet with an image [in each cell](https://xlsxwriter.readthedocs.io/worksheet.html#insert_image)

In [ ]:
sheet = writer.sheets['molecules']  # Get the sheet

In [ ]:
for i, smiles in enumerate(results['smiles']):  # Paste in each molecule
    sheet.insert_image(f'F{i+2}', 'smiles.png', {'image_data': molecule_to_png(smiles), 'object_position': 1})
    sheet.set_row_pixels(i + 1, 270)

Make the column wide enough to fit the image

In [ ]:
sheet.set_column_pixels(5, 5, 270)

Write it to disk!

In [ ]:
writer.save()